# `LlamaCpp` examples

This notebook contains usage examples of the llama-cpp-python support in guidance.
The first example is RPG character json generator.
The second example takes a jsonformer scheme as input and generate a valid json.
The third example uses the chat mode of guidance.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import guidance

# Create a LlamaCpp instance and pass the settings to it.
# note this assumes you have llama-cpp-python>=0.1.55 installed
llama = guidance.llms.LlamaCpp(
    model = "/Users/nielswarncke/models/gguf/llama-2-coder-7b.Q4_K_M.gguf", # downloaded from: https://huggingface.co/TheBloke/Llama-2-Coder-7B-GGUF/blob/main/llama-2-coder-7b.Q4_K_M.gguf
    tokenizer="mrm8488/llama-2-coder-7b",
    n_gpu_layers=10,
    n_threads=12
)

# Other models would look like this:
# model = "../ggml-v3-models/gpt4-x-vicuna-13B.ggmlv3.q5_1.bin"
# tokenizer = "TheBloke/gpt4-x-vicuna-13B-HF"

# Or like this:
# model = "../ggml-v3-models/Manticore-13B.ggmlv3.q5_1.bin"
# tokenizer = "openaccess-ai-collective/manticore-13b"

/Users/nielswarncke/miniconda3/envs/textgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/nielswarncke/models/gguf/llama-2-coder-7b.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_

In [14]:
# 1st Example
# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
character_maker = guidance("""The following is a character profile for an RPG game in JSON format.
```json
{
    "id": "{{id}}",
    "description": "{{description}}",
    "name": "{{gen 'name' temperature=0.7}}",
    "age": {{gen 'age' pattern='[0-9]+' max_tokens=2 stop=','}},
    "armor": "{{#select 'armor'}}leather{{or}}chainmail{{or}}plate{{/select}}",
    "weapon": "{{select 'weapon' options=valid_weapons}}",
    "class": "{{gen 'class'}}",
    "mantra": "{{gen 'mantra' temperature=0.7}}",
    "strength": {{gen 'strength' pattern='[0-9]+' max_tokens=2 stop=','}},
    "items": [{{#geneach 'items' num_iterations=5 join=', '}}"{{gen 'this' temperature=0.7}}"{{/geneach}}]
}```""")

# generate a character
char = character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons, llm=llama, caching=True
)

Stop program The following is a character profile for an RPG game in JSON format.
```json
{
 "id": " e1f491f7-7ab8-4dac-8c20-c92b5e7d883d ",
 "description": " A quick and nimble fighter. ",
 "name": "

{'chainmail': -1000, 'leather': -1000, 'plate': -1000}

Traceback (most recent call last):
  File "/Users/nielswarncke/Documents/agi/guidance/guidance/_program_executor.py", line 102, in run
    await self.visit(self.parse_tree)
  File "/Users/nielswarncke/Documents/agi/guidance/guidance/_program_executor.py", line 437, in visit
    visited_children.append(await self.visit(child, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/Users/nielswarncke/Documents/agi/guidance/guidance/_program_executor.py", line 437, in visit
    visited_children.append(await self.visit(child, inner_next_node, inner_next_next_node, inner_prev_node, node, parent_node))
  File "/Users/nielswarncke/Documents/agi/guidance/guidance/_program_executor.py", line 404, in visit
    command_output = await command_function(*positional_args, **named_args)
  File "/Users/nielswarncke/Documents/agi/guidance/guidance/library/_select.py", line 177, in select
    raise ValueError("No valid o

ValueError: No valid option generated in #select! Please post a GitHub issue since this should not happen :)

In [19]:
# 2nd Example
# define a recursive function that takes a jsonformer schema and returns a guidance program
def jsonformer2guidance(schema, key=None, indent=0):
    out = ""
    if schema['type'] == 'object':
        out += "  "*indent + "{\n"
        for k,v in schema['properties'].items():
            out += "  "*(indent+1) + k + ": " + jsonformer2guidance(v, k, indent+1) + ",\n"
        out += "  "*indent + "}"
        return out
    elif schema['type'] == 'array':
        if 'max_items' in schema:
            extra_args = f" max_iterations={schema['max_items']}"
        else:
            extra_args = ""
        return "[{{#geneach '"+key+"' stop=']'"+extra_args+"}}{{#unless @first}}, {{/unless}}" + jsonformer2guidance(schema['items'], "this") + "{{/geneach}}]"
    elif schema['type'] == 'string':
        return "\"{{gen '"+key+"' stop='\"'}}\""
    elif schema['type'] == 'number':
        return "{{gen '"+key+"' pattern='[0-9\\.]' stop=','}}"
    elif schema['type'] == 'boolean':
        return "{{#select '"+key+"'}}True{{or}}False{{/select}}"

In [20]:
# define a jsonformer schema
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "max_items": 6,
            "items": {"type": "string"}
        }
    }
}



In [21]:
# run the guidance program
prompt = "Generate a person's information based on the following schema:"
program = guidance(prompt + "\n" + jsonformer2guidance(json_schema))
out = program(llm=llama, silent=True)
print(out)

<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x10428c9d0>>
<bound method LlamaCpp.decode of <guidance.llms._llama_cpp.LlamaCpp object at 0x

ValueError: No valid option generated in #select! Please post a GitHub issue since this should not happen :)

In [14]:
# 3rd Example
# settings =  guidance.llms.LlamaCppSettings()
# settings.model = "../ggml-v3-models/Manticore-13B-Chat-Pyg.ggmlv3.q5_1.bin"
# settings.n_gpu_layers = 10
# settings.n_threads = 12
# settings.tokenizer_name = "openaccess-ai-collective/manticore-13b-chat-pyg"
# settings.before_role = "<|"
# settings.after_role = "|>"
llama = guidance.llms.LlamaCpp(
    model = "../ggml-v3-models/Manticore-13B-Chat-Pyg.ggmlv3.q5_1.bin",
    n_gpu_layers = 10,
    n_threads = 12,
    tokenizer = "openaccess-ai-collective/manticore-13b-chat-pyg",
    before_role = "<|",
    after_role = "|>",
    caching=False, silent=True
)


TypeError: __init__() got an unexpected keyword argument 'tokenizer_name'

In [24]:

# llama = guidance.llms.LlamaCpp(settings=settings, caching=False)
experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}
''', llm=llama)


experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{gen 'answer' temperature=0 max_tokens=500}}
''', llm=llama)


In [25]:
print(experts(query='How can I be more productive?', llm=llama))

Stop program system You are a helpful and terse assistant. </system>

<question>What is the name of the system?

</question>

<answer>The system's name is "System".

</answer>

<question>How can you access the system?

</question>

<answer>To access the system, use the command "system" followed by the desired action (e.g., "system list", "system create").

</answer>

<question>What actions can be performed on the system?

</question>

<answer>Actions that can be performed on the system include creating new systems, listing existing systems, deleting existing systems, and modifying existing systems.

</answer>

<question>Is there any documentation available for the system?

</question>

<answer>Yes, there is documentation available for the system. The documentation can be found at <a href="https://www.example.com/documentation">https://www.example.com/documentation</a>.

</answer>

<question>Are there any known issues with the system?

</question>

<answer>No, there are no known issues with the system.

</answer>


<system>You are a helpful and terse assistant.

</system>

<question>What is the name of the system?

</question>

<answer>The system's name is "System".

</answer>

<question>How can you access the system?

</question>

<answer>To access the system, use the command "system" followed by the desired action (e.g., "system list", "system create").

</answer>

<question>What actions can be performed on the system?

</question>

<answer>Actions that can be performed on the system include creating new systems, listing existing systems, deleting existing systems, and modifying existing systems.

</answer>

<question>Is there any documentation available for the system?

</question>

<answer>Yes, there is documentation available for the system. The documentation can be found at <a href="https://www.example.com/documentation">https://www.example.com/documentation</a>.

</answer>

<question>Are there any known issues with the system?

</question>

<answer>No, there are no known issues with the s